In [1]:
import qepy
import numpy as np

In [2]:
try:
    from mpi4py import MPI
    comm=MPI.COMM_WORLD
except:
    comm=None

In [3]:
from qepy.calculator import QEpyCalculator
import ase.io
from ase.io.trajectory import Trajectory
from ase import units
from ase.md.andersen import Andersen
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

In [4]:
np.random.seed(2020)
inputfile = 'qe_in.in'

calc = QEpyCalculator(comm = comm, inputfile = inputfile, logfile='tmp.out')
atoms = ase.io.read(inputfile, format='espresso-in')
atoms.set_calculator(calc)

T = 300
MaxwellBoltzmannDistribution(atoms, temperature_K = T, force_temp=True)
dyn = Andersen(atoms, 1.0 * units.fs, temperature_K = T, andersen_prob=0.02)

In [5]:
step = 0
interval = 1

def printenergy(a=atoms):
    global step, interval
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)
    if a.calc.is_root :
        print("Step={:<8d} Epot={:.5f} Ekin={:.5f} T={:.3f} Etot={:.5f}".format(
                step, epot, ekin, ekin / (1.5 * units.kB), epot + ekin, flush=True))
    step += interval

In [6]:
class WF(object):
    def __init__(self, calc=None, kpt=0, band=3):
        self.calc=calc
        self.kpt=kpt
        self.band=band
        self.wf_prev=None
        self.wf=None
    def __call__(self, i=0, j=0):
        wf = self.calc.get_wave_function(kpt=self.kpt, band=self.band)
        if self.wf is None:
            self.wf = wf
            self.wf_prev = wf
        else:
            self.wf, self.wf_prev = self.wf_prev, self.wf
            self.wf = wf
        result=(self.wf_prev[i]*self.wf[j].conj()).sum()/len(self.wf[j])
        
        if comm:
            result=comm.bcast(result)
            if comm.rank==0:
                print('WF:', result, flush=True)
        else:
            print('WF:', result, flush=True)
        return result

In [7]:
wf=WF(calc=calc, kpt=0, band=[3,4])
traj = Trajectory("tmp.traj", "w", atoms)
dyn.attach(printenergy, interval=1)
dyn.attach(traj.write, interval=1)
dyn.attach(wf, interval=1)
dyn.run(5)

Step=0        Epot=-154.42176 Ekin=0.03878 T=300.000 Etot=-154.38298
WF: (0.9999999999999996+0j)
Step=1        Epot=-154.42080 Ekin=0.01194 T=92.365 Etot=-154.40886
WF: (0.999972604995392-2.989861989057529e-19j)
Step=2        Epot=-154.41831 Ekin=0.01243 T=96.151 Etot=-154.40588
WF: (-0.9999673891125346-1.8655784510923428e-19j)
Step=3        Epot=-154.41095 Ekin=0.00588 T=45.528 Etot=-154.40506
WF: (0.9999717833887573-7.567070261569475e-19j)
Step=4        Epot=-154.41049 Ekin=0.00548 T=42.387 Etot=-154.40502
WF: (0.9999736867296904-2.9597743059436623e-19j)
Step=5        Epot=-154.41751 Ekin=0.01167 T=90.318 Etot=-154.40584
WF: (-0.9999704097482442-3.4501532954292265e-18j)


True